*Primer Pre entregable*  
*Militello Gastón*

In [1]:
import requests
from dotenv import load_dotenv
import os
import psycopg2

# Cargar variables de entorno desde el archivo .env
load_dotenv()

def get_weather(city):
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verificar errores HTTP
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

def insert_weather_data_into_redshift(data):
    try:
        # Conexión a Redshift
        conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )

        # Cursor para ejecutar comandos SQL
        cur = conn.cursor()

        # Insertar los datos meteorológicos en la tabla de Redshift
        query = """
            INSERT INTO weather_data (city, temperature, humidity)
            VALUES (%s, %s, %s)
        """
        cur.execute(query, (data['name'], data['main']['temp'], data['main']['humidity']))

        # Guardar los cambios en la base de datos
        conn.commit()

        print("Datos meteorológicos insertados en Redshift correctamente.")
    except Exception as e:
        print("Error al insertar datos en Redshift:", e)
    finally:
        # Cerrar el cursor y la conexión
        cur.close()
        conn.close()

def main():
    city = "Buenos Aires"  # Nombre de la ciudad para los datos
    weather_data = get_weather(city)
    
    if weather_data:
        print("Datos meteorológicos:", weather_data)
        insert_weather_data_into_redshift(weather_data)
    else:
        print("No se pudieron recuperar los datos meteorológicos.")

if __name__ == "__main__":
    main()

Datos meteorológicos: {'coord': {'lon': -58.3772, 'lat': -34.6132}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 25.24, 'feels_like': 26.14, 'temp_min': 24.4, 'temp_max': 26.66, 'pressure': 1004, 'humidity': 89}, 'visibility': 10000, 'wind': {'speed': 6.71, 'deg': 110, 'gust': 0}, 'clouds': {'all': 75}, 'dt': 1710882557, 'sys': {'type': 2, 'id': 2092396, 'country': 'AR', 'sunrise': 1710842173, 'sunset': 1710885963}, 'timezone': -10800, 'id': 3435910, 'name': 'Buenos Aires', 'cod': 200}
Datos meteorológicos insertados en Redshift correctamente.
